In [4]:
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil
from function import *

## dark特定の事前準備

処理手順  
ダークの名前を読み込むためのリスト作成   
ダークを特定して、データフレームに必要な情報を入れる  
そのcsvを該当の年のフォルダに移す  

In [20]:
def searchdarkfile(year):  #first
    basefolder = r'/mnt/e/DataFrame/'
    allframe = pd.read_excel(basefolder + "COM_"+year+".xlsx", sheet_name="COM_"+year,skiprows=0)
    allframe.dropna(how="all", axis=0)
    darkframe = allframe[(allframe["DATA_TYP"].str.contains("DARK")) |
                        (allframe["OBJECT2"].str.contains("DARK"))]
    darkframe = darkframe.reset_index(drop=True)
    darkframe.to_csv(r'/mnt/e/DataFrame/dark/darkframe_' +
                    year + r'.csv', encoding="utf-8")
    return darkframe

In [19]:
allframe = pd.read_excel(r'/mnt/e/DataFrame/COM_2009.xlsx',sheet_name="COM_2009")
allframe.dropna(how="all",axis=0)
allframe

,#FRAME_ID,DATE_OBS,FITS_SIZE[MB],FILTER01,FILTER02,FILTER03,FILTER04,RA2000,2000-12-01 00:00:00,GALLONG,...,ECLLAT,X_2000,Y_2000,Z_2000,UT_STR,EXPTIME,ALTITUDE,DATA_TYP,OBS_MOD,OBJECT2
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,COMA00096823,2009-01-13,2.0,H13,H21,F39C20.50W1.00,L02L20I,23:03:44.982000,+28:04:48.05,95.74157,...,31.14058,0.855850,-0.214380,0.470704,04:22:31,0.030,55.83240,OBJECT,imaging,beta_Peg
2,COMA00096825,2009-01-13,2.0,H13,H21,F39C20.50W1.00,L02L20I,23:03:46.435000,+28:04:58.02,95.74848,...,31.14063,0.855850,-0.214284,0.470747,04:25:06,0.030,55.25119,OBJECT,imaging,beta_Peg
3,COMA00096827,2009-01-13,1.0,H13,H21,F39C20.50W1.00,L02L20I,23:03:46.052000,+28:04:53.40,95.74635,...,31.14013,0.855854,-0.214311,0.470727,04:27:03,0.031,54.93959,OBJECT,imaging,beta_Peg
4,COMA00096829,2009-01-13,0.0,H13,H21,F39C20.50W1.00,L02L20I,23:03:46.053000,+28:04:53.36,95.74634,...,31.14012,0.855855,-0.214311,0.470727,04:28:36,0.021,54.59797,OBJECT,imaging,beta_Peg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4364,COMA00103801,2009-12-28,7.0,D12,H21,F37C18.75W0.85,L02L20I,13:11:35.998000,+19:53:07.86,336.51926,...,25.27979,-0.894855,-0.289030,0.340142,17:07:19,0.100,89.93960,DARK,imaging,DARK
4365,COMA00103803,2009-12-28,1.0,D12,H21,F37C18.75W0.85,L02L20I,13:12:19.637000,+19:53:07.66,337.48476,...,25.35130,-0.893933,-0.291869,0.340141,17:08:08,0.010,89.93960,DARK,imaging,DARK
4366,COMA00103804,2009-12-28,2.0,D12,H21,F37C18.75W0.85,L02L20I,13:12:19.637000,+19:53:07.66,337.48476,...,25.35130,-0.893933,-0.291869,0.340141,17:08:08,0.301,89.93960,DARK,spectroscop,y DARK
4367,COMA00103805,2009-12-28,1.0,D12,H21,F37C18.75W0.85,L02L20I,13:13:19.426000,+19:53:07.39,338.77608,...,25.44922,-0.892656,-0.295753,0.340140,17:09:02,0.010,89.93960,DARK,imaging,DARK


In [22]:
#2003年分からのもので作成
yearlist = [str(2008+i) for i in range(13)]
for year in yearlist:
    try:
        searchdarkfile(year)
    except:
        pass

COMAのファイルから、ダークのヘッダーを読み込み必要な情報を取得する。  
データフレームに格納する  
また、dark画像をdarkフォルダを作成して移動する。  

### メモ
INS_VER 2001-09-29以降のものにしか、YSTRTがなかった  
YSTRTの指定により、y軸のピクセル数が変わってくる  
1なら240pixだが、40なら160pixというように

In [8]:
def makedarkcsv2(year):
    basefolder = r'/mnt/e/'+year + r'/'
    darkdf = pd.DataFrame(
        columns=["FRAME_ID", "DATE", "PIXTIM", "RRSTRT", "YSTRT", "CoAdd", "NAXIS1","NAXIS2","NAXIS3","NAXIS4","Q_CHEB",  "OBS-MOD", "Q_DTYPE","Q_DETST"])
    darkframe = pd.read_csv(r'/mnt/e/DataFrame/dark/darkframe_' +
                            year + r'.csv')
    for darkfile in darkframe["#FRAME_ID"]:
        # header読み込み
        darkfilepath = basefolder + darkfile + ".fits"
        darkheader = readheader(darkfilepath)
        darkdata = readdata(darkfilepath)

        list1 = [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"],
                darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"], darkheader["NAXIS1"],darkheader["NAXIS2"], darkheader["NAXIS3"],darkheader["NAXIS4"],darkheader["Q_CHEB"],darkheader["OBS-MOD"], darkheader["Q_DTYPE"],darkheader["Q_DETST"]]
        add_row = pd.DataFrame([list1], columns=darkdf.columns)
        darkdf = pd.concat([darkdf, add_row], ignore_index=True)
        # dark画像を移動
        os.makedirs(basefolder+r'dark', exist_ok=True)
        shutil.move(darkfilepath, basefolder+r'dark/')
    darkdf.to_csv(basefolder+r'dark/dark.csv', index=False, encoding="utf-8")


In [9]:
makedarkcsv2("2003")

In [28]:
yearlist = [str(2000+i) for i in range(14)]
for year in yearlist:
    try:
        makedarkcsv(year)
    except:
        pass

In [ ]:
makedarkcsv("2003")

## 天体画像の入力に対してdarkを特定(dfを返す)

In [2]:
def identify(obsfilename):
    obsheader = readheader(obsfilename)
    #年を取得
    year = obsheader["DATE-OBS"][:4]
    #DARK.csvにアクセス
    darkcsv = r'/mnt/e/'+year +r'/dark/dark.csv'
    darkdf = pd.read_csv(darkcsv)
    #一致条件を指定する
    Q_PIXTIM = obsheader["Q_PIXTIM"]
    Q_RRSTRT = obsheader["Q_RRSTRT"]
    DATE = obsheader["DATE-OBS"]
    Q_YSTRT = obsheader["Q_YSTRT"]
    NAXIS4 = obsheader["NAXIS4"]
    A = (darkdf["PIXTIM"] == Q_PIXTIM)
    B = (darkdf["RRSTRT"] == Q_RRSTRT)
    C = (darkdf["DATE"] == DATE)
    D = (darkdf["YSTRT"] == Q_YSTRT)
    E = (darkdf["NAXIS4"] == NAXIS4)
    df = darkdf[A & B & C & D &E].reset_index(drop=True)
    return df, year

## 特定後、1Expあたりの平均画像作成

In [1]:

def make1Expdark(obsfile):
    df = identify(obsfile)[0]
    year = identify(obsfile)[1]
    darkbasefolder = r'/mnt/e/' + year +r'/dark/'
    obsdata= readdata(obsfile)
    #obsdataと同じshapeの0のnumpyを作成
    
    #RAWモードの場合に、この時点で、z方向をQ_CHEBごとに和をとるのもいいかもしれない。
    """
    if obsheader["Q_CHAM"] == 0:
        後で考える。
    """
    darkdata = np.zeros(list(obsdata.shape))
    if len(df) == 0:
        print("該当のダークが存在しない")
        #ここにはその場合の処理を記載する。
        
    #特定したダークファイル一枚一枚に対して、Q_CHEBが同じものを抽出??

    # df[df["Q_CHEB"].duplicated()]
    for darkfile in df["FRAME_ID"]:
        darkfilepath = darkbasefolder + darkfile +r'.fits'
        darkdata += np.mean(readdata(darkfilepath),axis=1,keepdims=True)
    #meanをとる
    meandark = darkdata / len(df)
    #1Expあたりに直す
    meandark_1 = meandark / int(df["Q_CHEB"][0])
    return meandark_1

検証用  
使用するのは、2003年の11/12にとられたCOMA00042689.fitsである。HD1255

In [9]:
obsf = r'/mnt/e/2003/COMA00042689.fits'

In [11]:
make1Expdark(obsf)

KeyError: 0